# Project authors: Cezary Suchorski, Michał Żarnowski

Graph dataset source: [Facebook Large Page-Page Network](https://snap.stanford.edu/data/facebook-large-page-page-network.html)

In [5]:
%pip install torch
%pip install torch-geometric
%pip install numpy pandas plotly scikit-learn tqdm umap-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.6 MB/s eta 0:00:00
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 40.5 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 34.8 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [2]:
import torch
from torch_geometric.datasets import FacebookPagePage
from torch_geometric.nn import Node2Vec

dataset = FacebookPagePage(root='/tmp/FaceBookPagePage')
data = dataset[0]



In [6]:
model = Node2Vec(data.edge_index, embedding_dim=128, walk_length=20, context_size=10, walks_per_node=10, num_negative_samples=1, p=1, q=1, sparse=True)
loader = model.loader(batch_size=128, shuffle=True)
optimizer = torch.optim.SparseAdam(model.parameters(), lr=0.01)

def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw, neg_rw)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [ ]:
for epoch in range(1, 101):
    loss = train()
    print(f'Epoch {epoch:03d}, Loss: {loss:.4f}')

z = model.embedding.weight
print(z)

KeyboardInterrupt: 